In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score
from sentence_transformers import SentenceTransformer
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import pipeline
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/df_selected.csv")

In [ ]:
X = df["Text"]
y = df["Score"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
y_train = y_train - np.min(y_train)
y_test = y_test - np.min(y_test)

In [ ]:
X_train.to_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/X_train.csv",index=False)
X_test.to_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/X_test.csv",index=False)
y_train.to_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/y_train.csv",index=False)
y_test.to_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/y_test.csv",index=False)

In [ ]:
X_train = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/X_train.csv")
X_test = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/X_test.csv")
y_train = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/y_train.csv")
y_test = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/data/y_test.csv")

In [ ]:
#X_train = X_train[0:10]

In [ ]:
#########################

In [ ]:
import torch
from transformers import pipeline
from tqdm import tqdm

# GPU kullanılabilirliğini kontrol et
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Flan-T5 large modelini yükle (GPU varsa GPU'yu kullan)
generator = pipeline("text2text-generation", model="google/flan-t5-large", device=device)

# X_train içindeki her cümle için prompt oluştur
prompts = [f"Paraphrase the following text while maintaining its original meaning: {text}" for text in X_train]

# Model ile metin üretimi
results = generator(
    prompts,
    max_length=120,
    num_return_sequences=4,  # Her metin için 4 farklı varyasyon üret
    do_sample=True,
    top_k=100,  # Daha geniş bir kelime havuzu
    top_p=0.95,  # Olasılığı daha yüksek kelimeler
    temperature=1.0,  # Daha çeşitli çıktılar almak için
    repetition_penalty=1.3,  # Tekrarı azaltmak için
    early_stopping=True
)

# Sonuçları yazdırma işlemi
for i, original in enumerate(X_train):
    print(f"Original: {original}")
    for j in range(4):
        print(f"Augmented {j+1}: {results[i][j]['generated_text']}")
    print("\n" + "-"*80 + "\n")





In [ ]:
# Veriyi organize etme
augmented_data = []

for i, original in enumerate(X_train):
    paraphrases = [results[i][j]['generated_text'] for j in range(4)]
    augmented_data.append({
        "original": original,
        "augmented_1": paraphrases[0],
        "augmented_2": paraphrases[1],
        "augmented_3": paraphrases[2],
        "augmented_4": paraphrases[3]
    })

    # Pandas DataFrame oluştur
df_augmented = pd.DataFrame(augmented_data)

In [ ]:
#Bu eski data
'''df_augmented.to_csv("/content/drive/MyDrive/Müş/LLM_Aug/T5/t5_augmented.csv",index=False)
df_augmented = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/T5/t5_augmented.csv")
df_augmented'''

In [ ]:
df_augmented = pd.read_csv("/content/drive/MyDrive/Müş/LLM_Aug/T5/t5_augmented.csv")

In [ ]:
df_augmented

In [ ]:
df_2x = pd.concat([df_augmented[['original']], df_augmented[['augmented_1']].rename(columns={"augmented_1": "original"})], ignore_index=True)
y_expanded = pd.concat([y_train, y_train], ignore_index=True)
# Yeni sütunu df_2x'e ekleyelim
df_2x["y"] = y_expanded.values
df_2x

In [ ]:
df_3x = pd.concat([
    df_augmented[['original']],
    df_augmented[['augmented_1']].rename(columns={"augmented_1": "original"}),
    df_augmented[['augmented_2']].rename(columns={"augmented_2": "original"})
], ignore_index=True)

# y_train etiketlerini tekrarlayarak ekle
y_expanded = pd.concat([y_train, y_train, y_train], ignore_index=True)
df_3x["y"] = y_expanded.values
df_3x

In [ ]:
df_5x = pd.concat([
    df_augmented[['original']],
    df_augmented[['augmented_1']].rename(columns={"augmented_1": "original"}),
    df_augmented[['augmented_2']].rename(columns={"augmented_2": "original"}),
    df_augmented[['augmented_3']].rename(columns={"augmented_3": "original"}),
    df_augmented[['augmented_4']].rename(columns={"augmented_4": "original"})
], ignore_index=True)

# y_train değerlerini 5 kez tekrar ederek ekle
y_expanded = pd.concat([y_train] * 5, ignore_index=True)

# Yeni y sütununu ekle
df_5x["y"] = y_expanded.values
df_5x

In [ ]:
y_2x_train=df_2x["y"]
y_3x_train=df_3x["y"]
y_5x_train=df_5x["y"]

# Embeddings

In [ ]:
X_test_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(X_test.tolist(),batch_size=32,show_progress_bar=True)

In [ ]:
X_original_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(X_train.tolist(),batch_size=32,show_progress_bar=True)

In [ ]:
X_2x_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(df_2x["original"].tolist(),batch_size=32,show_progress_bar=True)

In [ ]:
X_3x_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(df_3x["original"].tolist(),batch_size=32,show_progress_bar=True)

In [ ]:
X_5x_train_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2").encode(df_5x["original"].tolist(),batch_size=32,show_progress_bar=True)

## Model Eğitimi

In [ ]:
#Original Data
# XGBoost modelini oluştur
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_original_train_embeddings, y_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))

In [ ]:
#2x Data
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_2x_train_embeddings, y_2x_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))

In [ ]:
#3x Data
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_3x_train_embeddings, y_3x_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))

In [ ]:
#5x Data
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Çok sınıflı sınıflandırma için
    num_class=len(np.unique(y)), # Sınıf sayısını belirle
    eval_metric='mlogloss',      # Çok sınıflı log loss metriği
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    use_label_encoder=False
)

# Modeli eğit
xgb_model.fit(X_5x_train_embeddings, y_5x_train)
# Test seti ile tahmin yap
y_pred = xgb_model.predict(X_test_embeddings)

# Model doğruluğunu hesapla
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

f1score = f1_score(y_test, y_pred,average="macro")
print(f"F1-Score: {f1score:.2f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0","1","2","3","4"], yticklabels=["0","1","2","3","4"])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Sınıflandırma raporunu yazdır
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))